<a href="https://colab.research.google.com/github/esraa9322/Berlin-BCI-Competition/blob/main/Berlin_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Imports and installation***

In [ ]:
%qtconsole

In [ ]:
! pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.7 MB/s eta 0:00:00


In [ ]:
import scipy.io
import pandas as pd
import numpy as np
import mne

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

# **Functions to be used**

In [ ]:
def discrete_fourier_transform(x):
    N = len(x)
    n = np.arange(N)
    k = n.reshape((N, 1))
    e = np.exp(-2j * np.pi * k * n / N)
    coeff = np.dot(e, x)
    return coeff

In [ ]:
def calculate_magnitude(coeff):
  N= len(coeff)
  magnitude= np.abs(coeff)/N 
  return magnitude

In [ ]:
def preprocess_Signal(person_data):

  sfreq = person_data['nfo']['fs'][0][0][0][0]
  EEGSignal   = person_data['cnt'].T
  nchannels, nsamples = EEGSignal.shape
  Y=person_data['mrk'][0][0][1]
  
  #chan_names = [s[0] for s in Signal['nfo']['clab'][0][0][0]]
  event_onsets  = person_data['mrk'][0][0][0]
  event_codes   = person_data['mrk'][0][0][1]

  labels = np.zeros((1, nsamples), int)
  labels[0, event_onsets] = event_codes

  cl_lab = [s[0] for s in person_data['nfo']['classes'][0][0][0]]
  cl1    = cl_lab[0]
  cl2    = cl_lab[1]
   
  trials = {}
  win = np.arange(int(0.5*sfreq), int(2.5*sfreq))
  nsamples = len(win)
  for cl, code in zip(cl_lab, np.unique(event_codes)):  
    cl_onsets = event_onsets[event_codes == code]
    trials[cl] = np.zeros((nchannels, nsamples, len(cl_onsets)))
    
   
  for i, onset in enumerate(cl_onsets):
        trials[cl][:,:,i] = EEGSignal[:, win+onset]

  left_hand  = np.rollaxis(trials[cl1], 2, 0)  
  right_hand = np.rollaxis(trials[cl2], 2, 0)
  data = np.concatenate([left_hand, right_hand])
  Y = np.concatenate([-np.ones(left_hand.shape[0]),np.ones(right_hand.shape[0])])
    
  return data,Y



# **Person one data and model**

In [ ]:
person1_data = scipy.io.loadmat('BCICIV_calib_ds1a.mat')

In [ ]:
person1 , target_val = preprocess_Signal(person1_data)

In [ ]:
person1.shape

(200, 59, 200)

In [ ]:
target_val = np.asarray(target_val, dtype = 'int')
np.random.shuffle(target_val)
target_val

array([-1, -1,  1,  1, -1,  1,  1,  1,  1,  1, -1, -1, -1, -1,  1, -1,  1,
        1, -1,  1, -1, -1,  1, -1, -1, -1, -1, -1,  1,  1,  1, -1, -1,  1,
       -1,  1, -1,  1, -1,  1,  1, -1,  1,  1,  1,  1, -1,  1, -1, -1,  1,
       -1, -1, -1,  1,  1,  1, -1,  1, -1, -1, -1, -1,  1, -1, -1,  1, -1,
        1, -1,  1,  1, -1,  1, -1,  1, -1, -1,  1,  1, -1, -1,  1, -1, -1,
        1, -1, -1,  1,  1, -1, -1,  1, -1,  1, -1, -1,  1, -1,  1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,
        1,  1,  1, -1,  1,  1, -1, -1,  1,  1, -1, -1,  1, -1,  1,  1, -1,
        1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1,  1,
        1, -1,  1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1,  1,  1,
        1, -1, -1,  1, -1,  1,  1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,
        1, -1,  1,  1, -1, -1,  1,  1,  1, -1, -1,  1, -1])

In [ ]:
person1=np.average(person1,axis=1)

In [ ]:
person1.shape

(200, 200)

In [ ]:
person1_coeff=discrete_fourier_transform(person1)
person1_coeff= person1_coeff.real

In [ ]:
clf = svm.SVC(kernel='linear', C=1, random_state=42)
scoresa = cross_val_score(clf, person1_coeff, target_val, cv=5)
scoresa

array([0.475, 0.425, 0.525, 0.4  , 0.45 ])

In [ ]:
classifier2 = KNeighborsClassifier(n_neighbors=3)
scoresb = cross_val_score(classifier2, person1_coeff, target_val, cv=5)
scoresb

array([0.525, 0.5  , 0.6  , 0.625, 0.475])

# **person two data and model**

In [ ]:
person2_data = scipy.io.loadmat('BCICIV_calib_ds1b.mat')

In [ ]:
person2 , target_val2 = preprocess_Signal(person2_data)

In [ ]:
target_val2 = np.asarray(target_val2, dtype = 'int')
np.random.shuffle(target_val2)
target_val2

array([-1,  1, -1, -1,  1, -1, -1,  1,  1,  1, -1, -1, -1, -1, -1,  1, -1,
        1, -1,  1,  1, -1,  1,  1,  1, -1, -1,  1, -1,  1, -1, -1, -1, -1,
       -1, -1,  1,  1, -1,  1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1, -1,
        1,  1,  1,  1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1,
       -1, -1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1,  1,  1, -1,
       -1,  1,  1, -1, -1, -1,  1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1,
       -1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1, -1,  1,  1,  1,
        1, -1, -1, -1, -1,  1, -1, -1,  1,  1, -1,  1,  1, -1, -1,  1,  1,
        1,  1, -1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1, -1, -1, -1, -1,
       -1,  1, -1,  1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,
       -1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1, -1, -1, -1,
        1, -1, -1, -1,  1, -1,  1,  1,  1,  1, -1,  1, -1])

In [ ]:
person2=np.average(person2,axis=1)

In [ ]:
person2_coeff=discrete_fourier_transform(person2)
person2_coeff= person2_coeff.real

In [ ]:
clf2 = svm.SVC(kernel='linear', C=1, random_state=42)
scores2a = cross_val_score(clf2, person2_coeff, target_val2, cv=5)
scores2a

array([0.55 , 0.475, 0.65 , 0.45 , 0.575])

In [ ]:
classifier2 = KNeighborsClassifier(n_neighbors=3)
scores2b = cross_val_score(classifier2, person2_coeff, target_val2, cv=5)
scores2b

array([0.625, 0.475, 0.625, 0.525, 0.675])

# **Person three data and model**

In [ ]:
person3_data = scipy.io.loadmat('BCICIV_calib_ds1c.mat')


In [ ]:
person3 , target_val3 = preprocess_Signal(person3_data)

In [ ]:
target_val3 = np.asarray(target_val3, dtype = 'int')
np.random.shuffle(target_val3)
target_val3

array([-1, -1, -1, -1,  1, -1, -1,  1, -1, -1,  1, -1,  1, -1,  1, -1,  1,
       -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1,  1,  1,  1, -1,
       -1,  1,  1,  1,  1,  1, -1,  1, -1, -1,  1,  1, -1, -1,  1, -1,  1,
       -1,  1,  1, -1,  1, -1, -1, -1,  1, -1,  1,  1, -1,  1,  1, -1, -1,
       -1,  1, -1, -1, -1,  1, -1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,
        1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,
        1, -1, -1,  1, -1,  1,  1, -1,  1, -1,  1, -1, -1, -1, -1,  1, -1,
       -1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1,  1,
        1, -1, -1, -1, -1,  1, -1, -1,  1,  1,  1,  1, -1,  1, -1, -1, -1,
       -1,  1,  1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1,
        1,  1,  1,  1,  1, -1, -1, -1,  1,  1, -1, -1, -1, -1,  1,  1, -1,
        1, -1,  1, -1,  1,  1,  1,  1, -1, -1, -1,  1, -1])

In [ ]:
person3 = np.average(person3,axis=1)

In [ ]:
person3_coeff=discrete_fourier_transform(person3)
person3_coeff= person3_coeff.real

In [ ]:
clf3 = svm.SVC(kernel='linear', C=1, random_state=42)
scores3a = cross_val_score(clf3, person3_coeff, target_val3, cv=5)
scores3a

array([0.325, 0.45 , 0.55 , 0.45 , 0.375])

In [ ]:
classifier3 = KNeighborsClassifier(n_neighbors=3)
scores3b = cross_val_score(classifier3, person3_coeff, target_val3, cv=5)
scores3b

array([0.325, 0.475, 0.45 , 0.45 , 0.4  ])

# **person 4 data and model**

In [ ]:
person4_data = scipy.io.loadmat('BCICIV_calib_ds1d.mat')

In [ ]:
person4 , target_val4 = preprocess_Signal(person4_data)

In [ ]:
target_val4 = np.asarray(target_val4, dtype = 'int')
np.random.shuffle(target_val4)
target_val4

array([-1, -1, -1, -1, -1,  1,  1, -1, -1,  1, -1, -1, -1,  1, -1, -1,  1,
        1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,
       -1, -1, -1,  1,  1, -1, -1, -1, -1,  1,  1,  1, -1,  1, -1,  1,  1,
       -1,  1,  1, -1, -1,  1,  1,  1,  1, -1,  1, -1,  1, -1, -1, -1, -1,
       -1, -1, -1,  1, -1, -1,  1, -1,  1, -1,  1,  1,  1,  1, -1,  1, -1,
       -1, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1,  1, -1,  1,  1,  1,  1,  1, -1, -1,  1,  1, -1,
        1, -1,  1, -1, -1, -1, -1, -1,  1, -1,  1, -1,  1,  1, -1, -1, -1,
       -1,  1,  1,  1,  1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1,  1,  1,
        1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1, -1, -1, -1,  1,
       -1,  1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1,  1, -1,  1,  1,  1,
       -1,  1, -1,  1, -1, -1,  1,  1,  1, -1, -1,  1,  1])

In [ ]:
person4 = np.average(person4,axis=1)

In [ ]:
person4_coeff=discrete_fourier_transform(person4)
person4_coeff= person4_coeff.real

In [ ]:
clf4 = svm.SVC(kernel='linear', C=1, random_state=42)
scores4a = cross_val_score(clf4, person4_coeff, target_val4, cv=5)
scores4a

array([0.45, 0.55, 0.7 , 0.55, 0.45])

In [ ]:
classifier4 = KNeighborsClassifier(n_neighbors=3)
scores4b = cross_val_score(classifier4, person4_coeff, target_val4, cv=5)
scores4b

array([0.475, 0.5  , 0.525, 0.65 , 0.475])

# **person 5 data and model**

In [ ]:
person5_data = scipy.io.loadmat('BCICIV_calib_ds1e.mat')

In [ ]:
person5 , target_val5 = preprocess_Signal(person5_data)

In [ ]:
target_val5 = np.asarray(target_val5, dtype = 'int')
np.random.shuffle(target_val5)
target_val5

array([-1,  1, -1, -1, -1,  1, -1,  1, -1, -1,  1, -1,  1,  1,  1, -1, -1,
       -1, -1,  1, -1,  1, -1,  1, -1, -1,  1,  1,  1, -1,  1, -1, -1,  1,
        1,  1, -1, -1, -1,  1, -1,  1,  1, -1,  1, -1,  1, -1, -1, -1,  1,
       -1,  1, -1, -1,  1, -1, -1, -1,  1,  1,  1,  1, -1, -1,  1,  1, -1,
       -1,  1, -1, -1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1, -1,
        1,  1,  1,  1, -1, -1, -1,  1,  1, -1, -1, -1, -1,  1, -1,  1, -1,
        1, -1,  1,  1, -1,  1,  1,  1,  1,  1, -1, -1,  1, -1, -1,  1, -1,
       -1,  1,  1,  1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,
        1,  1,  1, -1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1, -1,  1,
       -1, -1,  1,  1, -1, -1, -1,  1,  1,  1, -1,  1, -1, -1, -1, -1,  1,
       -1, -1, -1,  1,  1, -1,  1,  1, -1, -1,  1,  1,  1,  1, -1, -1,  1,
        1,  1,  1, -1,  1, -1,  1, -1, -1,  1,  1,  1, -1])

In [ ]:
person5 = np.average(person5,axis=1)

In [ ]:
person5_coeff=discrete_fourier_transform(person5)
person5_coeff= person5_coeff.real

In [ ]:
person5_mag= calculate_magnitude(person5_coeff)
person5_mag.shape

(200, 200)

In [ ]:
clf5 = svm.SVC(kernel='linear', C=1, random_state=42)
scores5a = cross_val_score(clf5, person5_mag, target_val5, cv=5)
scores5a

array([0.425, 0.45 , 0.45 , 0.475, 0.425])

In [ ]:
classifier5 = KNeighborsClassifier(n_neighbors=3)
scores5b = cross_val_score(classifier5, person5_mag, target_val5, cv=5)
scores5b

array([0.5  , 0.475, 0.55 , 0.45 , 0.45 ])

# **person 6 data and model**

In [ ]:
person6_data = scipy.io.loadmat('BCICIV_calib_ds1f.mat')

In [ ]:
person6 , target_val6 = preprocess_Signal(person6_data)

In [ ]:
target_val6 = np.asarray(target_val6, dtype = 'int')
np.random.shuffle(target_val6)
target_val6

array([ 1,  1, -1, -1,  1, -1,  1,  1, -1, -1,  1,  1,  1,  1, -1, -1, -1,
        1, -1,  1,  1,  1,  1, -1, -1, -1, -1,  1, -1,  1,  1,  1,  1, -1,
       -1,  1, -1, -1, -1,  1,  1,  1, -1, -1, -1,  1, -1,  1,  1,  1, -1,
        1,  1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1, -1,  1, -1,  1, -1,
        1, -1,  1,  1, -1, -1, -1,  1, -1, -1, -1,  1,  1,  1,  1,  1, -1,
       -1, -1, -1,  1,  1,  1, -1,  1,  1, -1,  1,  1, -1, -1, -1, -1,  1,
       -1, -1,  1, -1, -1, -1, -1,  1,  1,  1, -1, -1, -1, -1, -1, -1,  1,
       -1, -1, -1,  1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
        1, -1,  1, -1, -1,  1,  1, -1, -1,  1,  1, -1,  1, -1, -1, -1, -1,
       -1,  1,  1,  1,  1, -1, -1, -1, -1, -1,  1,  1,  1, -1,  1,  1, -1,
       -1,  1,  1, -1, -1,  1,  1,  1,  1, -1,  1, -1,  1,  1,  1,  1, -1,
       -1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1])

In [ ]:
person6 = np.average(person6,axis=1)

In [ ]:
person6_coeff=discrete_fourier_transform(person6)
person6_coeff= person6_coeff.real

In [ ]:
#nyquist = 50
person6_coeff_filtered = mne.filter.filter_data(data = person6_coeff, sfreq=100, l_freq=0.1, h_freq=26, picks = None)

Setting up band-pass filter from 0.1 - 26 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 26.00 Hz
- Upper transition bandwidth: 6.50 Hz (-6 dB cutoff frequency: 29.25 Hz)
- Filter length: 3301 samples (33.010 sec)



<ipython-input-50-c4f16354c568>:2: RuntimeWarning: filter_length (3301) is longer than the signal (200), distortion is likely. Reduce filter length or filter a longer signal.
  person6_coeff_filtered = mne.filter.filter_data(data = person6_coeff, sfreq=100, l_freq=0.1, h_freq=26, picks = None)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished


In [ ]:
clf6 = svm.SVC(kernel='linear', C=1, random_state=42)
scores6a = cross_val_score(clf6, person6_coeff_filtered, target_val6, cv=5)
scores6a


array([0.425, 0.5  , 0.525, 0.5  , 0.375])

In [ ]:
classifier6 = KNeighborsClassifier(n_neighbors=3)
scores6b = cross_val_score(classifier6, person6_coeff_filtered, target_val6, cv=5)
scores6b


array([0.475, 0.525, 0.55 , 0.4  , 0.4  ])

# **Person 7 data and model**

In [ ]:
person7_data = scipy.io.loadmat('BCICIV_calib_ds1g.mat')

In [ ]:
person7 , target_val7 = preprocess_Signal(person7_data)

In [ ]:
target_val7 = np.asarray(target_val7, dtype = 'int')
np.random.shuffle(target_val7)
target_val7

array([ 1, -1,  1, -1,  1,  1,  1,  1,  1,  1, -1, -1, -1,  1,  1,  1,  1,
       -1, -1, -1, -1,  1,  1,  1,  1, -1, -1, -1,  1, -1, -1, -1,  1,  1,
       -1,  1, -1, -1, -1,  1, -1, -1,  1,  1,  1, -1, -1, -1, -1, -1,  1,
       -1,  1,  1, -1,  1, -1, -1,  1, -1,  1, -1,  1,  1,  1,  1, -1,  1,
        1,  1, -1,  1,  1, -1,  1,  1,  1, -1, -1,  1, -1, -1, -1, -1,  1,
        1, -1, -1, -1, -1,  1,  1,  1, -1, -1,  1,  1,  1, -1, -1, -1, -1,
       -1, -1,  1,  1,  1,  1, -1,  1, -1,  1,  1,  1,  1, -1,  1, -1, -1,
       -1,  1,  1, -1, -1,  1,  1, -1, -1,  1, -1, -1, -1,  1,  1, -1, -1,
       -1,  1, -1,  1,  1, -1, -1, -1, -1,  1, -1,  1,  1, -1,  1, -1, -1,
       -1, -1, -1,  1, -1,  1,  1,  1,  1,  1, -1,  1, -1,  1, -1,  1,  1,
        1,  1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,
        1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1])

In [ ]:
person7 = np.average(person7,axis=1)

In [ ]:
person7_coeff=discrete_fourier_transform(person7)
person7_coeff= person7_coeff.real

In [ ]:
person7_coeff_filtered = mne.filter.filter_data(data = person7_coeff, sfreq=100, l_freq=8, h_freq=40, picks = None)

Setting up band-pass filter from 8 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 165 samples (1.650 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished


In [ ]:
clf7 = svm.SVC(kernel='linear', C=1, random_state=42)
scores7a = cross_val_score(clf7, person7_coeff_filtered, target_val7, cv=5)
scores7a

array([0.45 , 0.725, 0.65 , 0.725, 0.45 ])

In [ ]:
classifier6 = KNeighborsClassifier(n_neighbors=3)
scores7b = cross_val_score(classifier6, person6_coeff_filtered, target_val7, cv=5)
scores7b

array([0.325, 0.675, 0.45 , 0.55 , 0.45 ])

# **Conclusion and notes**

Models on persons from 1 to 4 use the sama approach as we calculate fourier transform for the whole signal , get the coeffeients , get the average on channels , then we train the model 
Model on person 5 calculates the magnitude using fourier coeffients , then we train the model on the magnitude and target
Model 6 and 7 we used bandpass filtering on the FC after calculating them on different frequency ranges using sampling freq = 100 HZ

In [ ]:
# SVM results 
S1 = np.mean(scoresa)
S2 = np.mean(scores2a)
S3 = np.mean(scores3a)
S4 = np.mean(scores4a)
S5 = np.mean(scores5a)
S6 = np.mean(scores6a)
S7 = np.mean(scores7a)

In [ ]:
print("person 1 accuracy :" , S1)
print("person 2 accuracy :" , S2)
print("person 3 accuracy :" , S3)
print("person 4 accuracy :" , S4)
print("person 5 accuracy :" , S5)
print("person 6 accuracy :" , S6)
print("person 7 accuracy :" , S7)


person 1 accuracy : 0.55
person 2 accuracy : 0.52
person 3 accuracy : 0.43000000000000005
person 4 accuracy : 0.54
person 5 accuracy : 0.44499999999999995
person 6 accuracy : 0.465
person 7 accuracy : 0.6000000000000001


In [ ]:
# KNN results
S11 = np.mean(scoresb)
S22 = np.mean(scores2b)
S33 = np.mean(scores3b)
S44 = np.mean(scores4b)
S55 = np.mean(scores5b)
S66 = np.mean(scores6b)
S77 = np.mean(scores7b)

In [ ]:
print("person 1 accuracy :" , S11)
print("person 2 accuracy :" , S22)
print("person 3 accuracy :" , S33)
print("person 4 accuracy :" , S44)
print("person 5 accuracy :" , S55)
print("person 6 accuracy :" , S66)
print("person 7 accuracy :" , S77)

person 1 accuracy : 0.475
person 2 accuracy : 0.5349999999999999
person 3 accuracy : 0.42000000000000004
person 4 accuracy : 0.525
person 5 accuracy : 0.485
person 6 accuracy : 0.47000000000000003
person 7 accuracy : 0.49000000000000005


# **Failed tests**

In [ ]:
####################
person888_coeff_filtered = mne.filter.filter_data(data = person7_coeff, sfreq=100, l_freq=8, h_freq=12, picks = None)

Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 165 samples (1.650 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished


In [ ]:
clf8 = svm.SVC(kernel='linear', C=1, random_state=42)
scores8a = cross_val_score(clf8, person888_coeff_filtered, target_val7, cv=5)
scores8a